In [1]:
# Dependencies
import requests
import json
import pandas as pd
from config import api_key

In [2]:
# EPA API base URL
base_url = f"https://aqs.epa.gov/data/api/dailyData/byCBSA?email=rob_mm@msn.com&key={api_key}"

bdate=20230601; edate=20231231; #cbsa=16740

In [3]:
def pull_data(param, param_code, cbsa_code='00000'):
    # url = base_url + f"&param={param_code}&bdate=20240101&edate=20240229&state=37&county=183" # dailyData/byCounty 
    url = base_url + f"&param={param_code}&bdate={bdate}&edate={edate}&cbsa={cbsa_code}" # dailyData/byCBSA
    
    response = requests.get(url).json()#; print(url)

    if (response['Header'][0]['status'] == 'Failed'):
        print(f"Error: {response['Header'][0]['error']}")

    state = []; cbsa_code = []
    latitude = []; longitude = []
    date = []; parameter = []
    arithmetic_mean = []
    units_of_measure = []
    sample_duration_code = []

    for row in range(len(response["Data"])):
        date.append(response["Data"][row]["date_local"])
        state.append(response["Data"][row]["state"])
        cbsa_code.append(response["Data"][row]["cbsa_code"])
        latitude.append(response["Data"][row]["latitude"])
        longitude.append(response["Data"][row]["longitude"])
        parameter.append(response["Data"][row]["parameter"])
        arithmetic_mean.append(response["Data"][row]["arithmetic_mean"])
        units_of_measure.append(response["Data"][row]["units_of_measure"])
        sample_duration_code.append(response["Data"][row]["sample_duration_code"])

    # create DataFrame
    df = pd.DataFrame({
        "date": date,
        "state": state,
        "cbsa_code": cbsa_code,
        "latitude": latitude,
        "longitude": longitude,
        "sample_duration_code": sample_duration_code,
        "parameter": parameter,
        "arithmetic_mean": arithmetic_mean,
        "units_of_measure": units_of_measure
    })

    return df

In [4]:
# Merge DataFrames
def merge_dataframes(CO_df, SO2_df, NO2_df, PM10_df, PM25_df, Temperature_df, Humidity_df):
    # CO, SO2, NO2
    Oxides_df = CO_df.merge(SO2_df, how='outer', on=['date', 'state', 'cbsa_code', 'latitude', 'longitude', 'sample_duration_code'], suffixes=('_CO', '_SO2'))
    Oxides_df = Oxides_df.merge(NO2_df, how='outer', on=['date', 'state', 'cbsa_code', 'latitude', 'longitude', 'sample_duration_code'])

    # PM10 and PM2.5
    PM_df = PM10_df.merge(PM25_df, how='outer', on=['date', 'state', 'cbsa_code', 'latitude', 'longitude', 'sample_duration_code'], suffixes=('_PM10', '_PM25'))

    # Temperature and Humidity
    TH_df = Temperature_df.merge(Humidity_df, how='outer', on=['date', 'state', 'cbsa_code', 'latitude', 'longitude'], suffixes=('_Temperature', '_Humidity'))

    # Oxides, PM, TH
    tmp_df = Oxides_df.merge(PM_df, how='outer', on=['date', 'state', 'cbsa_code', 'latitude', 'longitude', 'sample_duration_code'])
    tmp_df = tmp_df.merge(TH_df, how='outer', on=['date', 'state', 'cbsa_code', 'latitude', 'longitude'])


    return tmp_df


In [5]:
def verify_uom(pollution_df):
    # Verify the Units of Measure (should show one row only)
    print(pollution_df.value_counts(subset=['cbsa_code', 'units_of_measure_CO',
                                    'units_of_measure_SO2',
                                    'units_of_measure',
                                    'units_of_measure_PM25',
                                    'units_of_measure_PM10',
                                    'units_of_measure_Temperature',
                                    'units_of_measure_Humidity'
                                    ]))


In [6]:
def get_PopulationDensity(cbsa_code):
    CBSA_df = pd.read_csv('USA_Core_Based_Statistical_Area.csv')

    CBSA_df = CBSA_df[['CBSA_ID', 'NAME', 'POP_SQMI']]

    CBSA_df.set_index('CBSA_ID', inplace=True)
    Pop_SqMi = CBSA_df.at[int(cbsa_code), 'POP_SQMI']
    Pop_SqKm = Pop_SqMi / 2.58998811
    # print(f"Pop_SqKm: {Pop_SqKm} people/km2")
    print(f"Pop_SqMi: {Pop_SqMi} people/mi2")

    return Pop_SqKm

In [7]:
get_PopulationDensity('16740')

Pop_SqMi: 449.01 people/mi2


173.36373022963411

In [8]:
def build_df(cbsa_code):
    CO_df = pull_data('CO', param_code=42101, cbsa_code=cbsa_code)
    CO_df = CO_df.drop_duplicates()

    SO2_df = pull_data('SO2', param_code=42401, cbsa_code=cbsa_code)
    SO2_df = SO2_df.drop_duplicates()

    NO2_df = pull_data('NO2', param_code=42602, cbsa_code=cbsa_code)
    NO2_df = NO2_df.drop_duplicates()

    PM10_df = pull_data('PM10', param_code=81102, cbsa_code=cbsa_code)
    PM10_df = PM10_df.drop_duplicates()

    PM25_df = pull_data('PM2.5', param_code=88101, cbsa_code=cbsa_code)
    PM25_df = PM25_df.drop_duplicates()

    Humidity_df = pull_data('Humidity', param_code=62201, cbsa_code=cbsa_code)
    Humidity_df = Humidity_df.drop_duplicates()

    Temperature_df = pull_data('Temperature', param_code=68105, cbsa_code=cbsa_code)
    Temperature_df = Temperature_df.drop_duplicates()
    
    pollution_df = merge_dataframes(CO_df, SO2_df, NO2_df, PM10_df, PM25_df, Temperature_df, Humidity_df)

    # get_PopulationDensity(cbsa_code)
    pollution_df.loc[:,'Population_Density'] = get_PopulationDensity(cbsa_code)
    pollution_df.loc[:,'Proximity_to_Industrial_Areas'] = 15 # made-up value, to complete the features
    
    # print(f'cbsa_code: {cbsa_code} done')

    return pollution_df

In [9]:
# drop duplicate rows
def drop_duplicates():
    CO_df = CO_df.drop_duplicates()
    SO2_df = SO2_df.drop_duplicates()
    NO2_df = NO2_df.drop_duplicates()
    PM10_df = PM10_df.drop_duplicates()
    PM25_df = PM25_df.drop_duplicates()
    Humidity_df = Humidity_df.drop_duplicates()
    Temperature_df = Temperature_df.drop_duplicates()

In [10]:
def take_important_columns_only(pollution_df):
    # take the important columns only
    pollution_df = pollution_df[['date',	'state', 'cbsa_code',
                        'arithmetic_mean_Temperature',
                        'arithmetic_mean_Humidity',
                        'arithmetic_mean_PM25',
                        'arithmetic_mean_PM10',
                        'arithmetic_mean',
                        'arithmetic_mean_SO2',
                        'arithmetic_mean_CO',
                        'Population_Density',
                        'Proximity_to_Industrial_Areas',
                        'latitude', 'longitude']]
    pollution_df = pollution_df.rename(columns={"arithmetic_mean_Temperature": "Temperature",
                                    "arithmetic_mean_Humidity": "Humidity",
                                    "arithmetic_mean_PM25": "PM2.5",
                                    "arithmetic_mean_PM10": "PM10",
                                    "arithmetic_mean": "NO2",
                                    "arithmetic_mean_SO2": "SO2",
                                    "arithmetic_mean_CO": "CO"})
    return pollution_df

In [11]:
from pathlib import Path

def get_cbsa_codes():
    filepath=Path('USA_Core_Based_Statistical_Area.geojson')
    with open(filepath) as jsonfile:
        geo_json = json.load(jsonfile)

    cbsa_code_list = []
    for i in range(len(geo_json['features'])):
        cbsa_code_list.append(geo_json['features'][i]['properties']['CBSA_ID'])

    return cbsa_code_list


In [12]:
pollution_df = pd.DataFrame()
cbsa_code_list = get_cbsa_codes()
for cbsa_code in cbsa_code_list[158:162]:
    pollutants_df = build_df(cbsa_code)
    
    verify_uom(pollutants_df)
    print(f"{cbsa_code}: {pollutants_df['cbsa_code'].count()} rows\n")
    
    pollutants_df = take_important_columns_only(pollutants_df)
    pollution_df = pd.concat([pollution_df, pollutants_df]).reset_index(drop=True)

Pop_SqMi: 449.01 people/mi2
cbsa_code  units_of_measure_CO  units_of_measure_SO2  units_of_measure   units_of_measure_PM25        units_of_measure_PM10          units_of_measure_Temperature  units_of_measure_Humidity
16740      Parts per million    Parts per billion     Parts per billion  Micrograms/cubic meter (LC)  Micrograms/cubic meter (25 C)  Degrees Centigrade            Percent relative humidity    68
Name: count, dtype: int64
16740: 3515 rows

Pop_SqMi: 143.38 people/mi2
Series([], Name: count, dtype: int64)
16820: 549 rows

Pop_SqMi: 264.19 people/mi2
Series([], Name: count, dtype: int64)
16860: 898 rows

Pop_SqMi: 37.54 people/mi2
cbsa_code  units_of_measure_CO  units_of_measure_SO2  units_of_measure   units_of_measure_PM25        units_of_measure_PM10          units_of_measure_Temperature  units_of_measure_Humidity
16940      Parts per million    Parts per billion     Parts per billion  Micrograms/cubic meter (LC)  Micrograms/cubic meter (25 C)  Degrees Centigrade           

In [13]:
filtered_df = pollution_df.dropna(how='any', ignore_index=True)

filtered_df.to_csv(f'EPA_Data_{bdate}to{edate}.csv', index=False)


In [18]:
# Get Max Values
filtered_df.groupby(['cbsa_code']).max(numeric_only=True)

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Population_Density,Proximity_to_Industrial_Areas,latitude,longitude
cbsa_code,,,,,,,,,,,
16740,29.0,96.416667,40.750000,50.583333,20.1250,0.841667,0.761458,173.363730,15,35.240100,-80.785683
16940,25.0,100.000000,9.666667,26.458333,7.9625,0.291304,0.219913,14.494275,15,41.182227,-104.778334


In [19]:
# Get Min Values
filtered_df.groupby(['cbsa_code']).min(numeric_only=True)

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Population_Density,Proximity_to_Industrial_Areas,latitude,longitude
cbsa_code,,,,,,,,,,,
16740,2.3,47.500000,1.625000,3.458333,1.390909,0.000000,0.14513,173.363730,15,35.240100,-80.785683
16940,-4.9,20.208333,2.095238,0.941176,0.795833,-0.008696,-0.03700,14.494275,15,41.182227,-104.778334


## For Cross-Checking Only

In [ ]:
# Use this to cross-check the source rows with filtered_df
df = PM25_df
df.loc[ (df['date'] == '2023-06-02')&
        (df['state'] == 'North Carolina')&
        (df['cbsa_code'] == 'Mecklenburg') &
        (df['sample_duration_code'] == '1') &
        (df['latitude'] == 35.2401) &
        (df['longitude'] == -80.785683)
    ]

In [ ]:
filtered_df.loc[filtered_df['date'] == '2023-06-02']

## Population Density

In [38]:
CBSA_df = pd.read_csv('USA_Core_Based_Statistical_Area.csv')

CBSA_df = CBSA_df[['CBSA_ID', 'NAME', 'POP_SQMI']]
# CBSA_df.head()

In [ ]:
def get_PopulationDensity(cbsa_code):
    CBSA_df = pd.read_csv('USA_Core_Based_Statistical_Area.csv')

    CBSA_df = CBSA_df[['CBSA_ID', 'NAME', 'POP_SQMI']]

    CBSA_df.set_index('CBSA_ID', inplace=True)
    Pop_SqMi = CBSA_df.at[cbsa_code, 'POP_SQMI']
    Pop_SqKm = Pop_SqMi / 2.58998811
    # print(f"Pop_SqKm: {Pop_SqKm} people/km2")
    print(f"Pop_SqMi: {Pop_SqMi} people/mi2")

    return Pop_SqKm

In [ ]:
# import numpy as np
# filtered_df.loc[:,'Population_Density'] = Pop_SqKm
# filtered_df.loc[:,'Proximity_to_Industrial_Areas'] = 15 # made-up value, to make dataset with complete features
# filtered_df
filtered_df.value_counts(subset=['cbsa_code', 'Population_Density', 'Proximity_to_Industrial_Areas'])

## Get all cbsa_code from geojson file

In [ ]:
from pathlib import Path

def get_cbsa_codes():
    filepath=Path('USA_Core_Based_Statistical_Area.geojson')
    with open(filepath) as jsonfile:
        geo_json = json.load(jsonfile)

    cbsa_code_list = []
    for i in range(len(geo_json['features'])):
        cbsa_code_list.append(geo_json['features'][i]['properties']['CBSA_ID'])

    return cbsa_code_list

# print(len(cbsa_code_list))
# geo_json['features'][158]['properties']['CBSA_ID']
# geo_json['features'][158]['properties']['NAME']
# features = geo_json['features']
# print(cbsa_code_list[0:21])